# Linear Regression Example Project
You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.
    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [ ]:
from pyspark.sql import *
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
# Use Spark to read in the Ecommerce Customers csv file.
df = sqlContext.read.format("com.databricks.spark.csv").options(header='true').options(inferSchema='true').load("file:/home/cloudera/Desktop/Python/MachineLearning/LinearRegression/cruise_ship_info.csv")


In [ ]:
# Print the Schema of the DataFrame
df.printSchema()

In [ ]:
df.show()

In [ ]:
df.columns

In [ ]:
df.groupby('Cruise_line').count().show()

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="crusie_cat")
indexed = indexer.fit(df).transform(df)

In [ ]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(
    inputCols=['Age','Tonnage', 'passengers',  'length',
 'cabins', 'passenger_density', 'crusie_cat'],
    outputCol="features")

In [ ]:
output = assembler.transform(indexed)
output.select("features").head(6)

In [ ]:
final_data = output.select("features",'crew')
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
train_data.describe().show()

In [ ]:
test_data.describe().show()

In [ ]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [ ]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [ ]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

In [ ]:
test_results = lrModel.transform(test_data)
test_results.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# evaluate RMSE
from math import sqrt
def calcRMSE(df, labelCol='crew'):
  """Given a DataFrame with 'label' and 'prediction' columns, calculate the root-mean-squared error
    Args:
        df: a DataFrame with labels and predictions        
    Returns:
        Float
  """
  return sqrt(float(df
                     .rdd
                     .map(lambda row: (row[labelCol]-row['prediction'])**2)
                     .reduce(lambda a,b: a+b)
                    ) / df.count() )

# evaluate R-squared
def calcRsq(df, labelCol='crew'):
  """Given a DataFrame with 'label' and 'prediction' columns, calculate the R-squared diagnostic  
    Args:
        df: a DataFrame with labels and predictions        
    Returns:
        Float
  """
  npts = df.count()
  mean_label = float(df.rdd.map(lambda row: row[labelCol]).reduce(lambda a,b: a+b))/npts
  total_squared_error = (df
                        .rdd
                        .map(lambda row: (row[labelCol]-mean_label)**2)
                        .reduce(lambda a,b: a+b)
                        )
  squared_error = (df
                   .rdd
                   .map(lambda row: (row[labelCol]-row['prediction'])**2)
                   .reduce(lambda a,b: a+b)
                  )  
  return (1.0 - float(squared_error) / total_squared_error)

print(calcRMSE(test_results))
print(calcRsq(test_results))